In [1]:
%matplotlib inline
import HElib
import torch
import NN
from torch import nn
import Chev
batch_size = 5
scale = 1000

In [2]:
HE = HElib.HE(w=2**19,l=50,eBound=1000,aBound=1000)
T = HE.TGen(batch_size,'cpu')
S = HE.getSecretkey(T)

In [3]:
x = torch.randn(batch_size,7*7*64).type(torch.float64)
xx = scale*x
c = HE.encrypt(x=xx,T=T)
fc1 = NN.Dense(in_features=7*7*64,out_features=1024).double()
fc1.build(HE,T,scale)
x1 = fc1(x)
c1 = fc1.crypted_forward(c)
x_1 = HE.decrypt(S=S,c=c1)/scale
(x1-x_1).mean()/x1.mean()

tensor(-0.0013, dtype=torch.float64, grad_fn=<DivBackward1>)

In [4]:
x = torch.randn(batch_size,32,28,28).type(torch.float64)
xx = scale*x
c = HE.encrypt(x=xx,T=T)
conv1 = NN.Conv(in_channels=32,out_channels=64,kernel_size=3,padding=1,stride=1,bias=True).double()
conv1.build(HE,T,scale)
x1 = conv1(x)
c1 = conv1.crypted_forward(c)
x_1 = HE.decrypt(S=S,c=c1)/scale
(x1-x_1).mean()/x1.mean()

tensor(0.0021, dtype=torch.float64, grad_fn=<DivBackward1>)

In [5]:
x = torch.randn(batch_size,32,28,28).type(torch.float64)
xx = scale*x
c = HE.encrypt(x=xx,T=T)
relu = NN.polynomial(coe=[1/4,1/2,1/4])
relu.build(HE=HE,T=T,scale=scale)
x1 = relu(x)
c1 = relu.crypted_forward(HE,c)
x_1 = HE.decrypt(S=S,c=c1)/scale
(x1-x_1).mean()/x1.mean()

tensor(-0.0030, dtype=torch.float64)

In [ ]:
net